In [1]:
# Reference
# https://yurimkoo.github.io/python/2019/09/14/connect-db-with-python.html

import pandas as pd
import requests
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
from functions.pubgapi import get_tournament_info
from functions.pubgapi import get_match_info
from functions.pubgapi import get_match_participant_single

# Load api key & RDS info (endpoint="rds 엔드포인트", dbname="db 이름", username="마스터 사용자 이름", password="rds 비밀번호")
load_dotenv()
api_key = os.environ.get("API_KEY")
endpoint = os.environ.get("AWS_endpoint")
dbname = os.environ.get('AWS_dbname')
username = os.environ.get("AWS_username")
password = os.environ.get("AWS_password")

In [13]:
# Tournament id 값 조회
tournament_info = get_tournament_info(api_key)
tournament_id = tournament_info["id"]

print(f"{len(tournament_id)} 개의 tournament_id 조회 완료\n")

# tournament_index를 지정하여 id값 출력
idx1 = 0
tournament_index_id = tournament_id[idx1]

cur_match_info = get_match_info(api_key, tournament_index_id)
num_match = len(cur_match_info["matchId"])
print(f"Tournament name: {tournament_index_id} ({num_match} matches)")

# Match participant 조회
idx2 = 2
match_info_id = cur_match_info["matchId"]
match_info_id_single = match_info_id[idx2]
match_participant = get_match_participant_single(api_key, match_info_id_single)
match_participant

467 개의 tournament_id 조회 완료

Tournament name: eu-tgs22 (5 matches)


,match_id,player_id,team_rank,dbnos,assists,boosts,damage_dealt,heals,kill_streaks,kills,longest_kill,revives,team_kills,time_survived,win
0,72a1c01f-5dd0-45a7-b6e5-224391eeffb6,BO_Perinho,9,0,0,4,94.344680,3,0,0,0.000000,0,0,1335,0
1,72a1c01f-5dd0-45a7-b6e5-224391eeffb6,BO_Haarfeste,9,0,0,5,57.142487,6,0,0,0.000000,0,0,297,0
2,72a1c01f-5dd0-45a7-b6e5-224391eeffb6,BO_SandrHeim,9,0,0,7,0.000000,3,0,0,0.000000,1,0,1335,0
3,72a1c01f-5dd0-45a7-b6e5-224391eeffb6,BO_Brutus,9,0,0,4,107.579310,6,0,0,0.000000,0,0,1294,0
4,72a1c01f-5dd0-45a7-b6e5-224391eeffb6,PF_OConnell,4,0,0,3,139.259720,1,0,0,0.000000,0,0,1364,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,72a1c01f-5dd0-45a7-b6e5-224391eeffb6,BZ_DAVETHECRAZY,16,0,0,0,76.394740,0,0,0,0.000000,0,0,715,0
59,72a1c01f-5dd0-45a7-b6e5-224391eeffb6,KTK_Yippikayey,14,0,1,2,113.816730,4,0,0,0.000000,1,0,989,0
60,72a1c01f-5dd0-45a7-b6e5-224391eeffb6,KTK_Doffy,14,1,1,2,333.575100,1,1,2,22.285328,0,0,986,0
61,72a1c01f-5dd0-45a7-b6e5-224391eeffb6,KTK_KINGDAINN,14,1,1,2,70.648360,4,1,1,25.272926,0,0,989,0


In [22]:
# DB와 connect

# create_engine("mysql+pymysql://아이디:"+"암호"+"@주소:포트/데이터베이스이름?charset=utf8", encoding='utf-8')
engine = create_engine("mysql+pymysql://admin:"+password+"@"+endpoint+":3306/"+dbname+"?charset=utf8", encoding="utf-8")
conn = engine.connect()

In [26]:
# sql = "DROP TABLE train_match_data"
# pd.read_sql(sql, conn)

In [18]:
# AWS RDS에 테이블 create

if match_participant["kills"].sum() != 0:
    # Kill 수의 합이 0인 경우 제외(연습게임으로 간주)
    match_participant.to_sql(name="train_match_data", con=conn, if_exists='append', index=False)

In [25]:
# 업로드된 테이블 확인
sql = "SHOW TABLES;"
result = pd.read_sql(sql, conn)
print(sql)
result

SHOW TABLES;


,Tables_in_jhk_pubg


In [20]:
# 내용 확인(SELECT *)
sql = f"SELECT * FROM train_match_data;"
result1 = pd.read_sql(sql, conn)
print(sql)
result1

SELECT * FROM train_match_data;


,match_id,player_id,team_rank,dbnos,assists,boosts,damage_dealt,heals,kill_streaks,kills,longest_kill,revives,team_kills,time_survived,win
0,72a1c01f-5dd0-45a7-b6e5-224391eeffb6,BO_Perinho,9,0,0,4,94.344680,3,0,0,0.000000,0,0,1335,0
1,72a1c01f-5dd0-45a7-b6e5-224391eeffb6,BO_Haarfeste,9,0,0,5,57.142487,6,0,0,0.000000,0,0,297,0
2,72a1c01f-5dd0-45a7-b6e5-224391eeffb6,BO_SandrHeim,9,0,0,7,0.000000,3,0,0,0.000000,1,0,1335,0
3,72a1c01f-5dd0-45a7-b6e5-224391eeffb6,BO_Brutus,9,0,0,4,107.579310,6,0,0,0.000000,0,0,1294,0
4,72a1c01f-5dd0-45a7-b6e5-224391eeffb6,PF_OConnell,4,0,0,3,139.259720,1,0,0,0.000000,0,0,1364,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,72a1c01f-5dd0-45a7-b6e5-224391eeffb6,BZ_DAVETHECRAZY,16,0,0,0,76.394740,0,0,0,0.000000,0,0,715,0
59,72a1c01f-5dd0-45a7-b6e5-224391eeffb6,KTK_Yippikayey,14,0,1,2,113.816730,4,0,0,0.000000,1,0,989,0
60,72a1c01f-5dd0-45a7-b6e5-224391eeffb6,KTK_Doffy,14,1,1,2,333.575100,1,1,2,22.285328,0,0,986,0
61,72a1c01f-5dd0-45a7-b6e5-224391eeffb6,KTK_KINGDAINN,14,1,1,2,70.648360,4,1,1,25.272926,0,0,989,0


In [21]:
# db접속 종료
conn.close()